In [ ]:
# make snapshot data form from separated time-series tables
# because of each patient has different number of records, each table has max record number of columns

In [33]:
import pandas as pd
from pathlib import Path
import yaml
import numpy as np
import datetime

In [20]:
# load pre-defined configurations
with open('../config.yaml')  as f:
    config = yaml.load(f, yaml.SafeLoader)

# table setting
table_name = 'clrc_oprt_nfrm'
columns = config['data_config']['required'][table_name.upper()]
prefix = config['data_config']['prefix'][table_name.upper()]

In [21]:
input_path = Path('/mnt/synthetic_data/data/raw/')

In [22]:
oprt_nfrm = pd.read_excel(input_path.joinpath('CLRC_OPRT_NFRM.xlsx'))

In [76]:
dead = pd.read_excel(input_path.joinpath('CLRC_DEAD_NFRM.xlsx'))

In [23]:
'''
transform to numpy form grouped by PT_SBST_NO
input : pandas dataframe
return : list of numpy array (2D-list like)
'''

def transform_format(data):
    
    input_form = []
    grouped = data.groupby(by ='PT_SBST_NO')
    
        
    uniq_pt_num = data['PT_SBST_NO'].unique()
    
    for pt in uniq_pt_num:
        input_form.append(grouped.get_group(pt).to_numpy())
        
        
    return input_form

In [24]:
max_record = oprt_nfrm['PT_SBST_NO'].value_counts().max() # max record in dataframe(table)

In [25]:
bsnf = pd.read_excel(input_path.joinpath('CLRC_PT_BSNF.xlsx')) # basic information table

In [26]:
np_oprt = transform_format(oprt_nfrm[list(columns)])

In [27]:
# select patinets who have opreation records
np_bsnf = bsnf[bsnf['PT_SBST_NO'].isin(oprt_nfrm['PT_SBST_NO'])].to_numpy()

In [34]:
# MAIN LOGIC
# flatten whole dataframe from discrete interval time series to snapshot

new_oprt_whole = []
for i in range(len(np_bsnf)):
    new_oprt = []
    for j in range(max_record):
        try:
            oprt_ymd = datetime.datetime.strptime(str(np_oprt[i][j][2]), '%Y%m%d')
            frst_diag_ymd = np_bsnf[i][list(bsnf.columns).index(bsnf.filter(like='YMD').columns[0])]
            frst_diag_ymd = datetime.datetime.strptime(str(frst_diag_ymd), '%Y%m%d')
            time_diff = (oprt_ymd - frst_diag_ymd).days
            remove_date = np.delete(np_oprt[i][j], 2, axis = 0)
            appended = np.append(remove_date, time_diff)
            new_oprt.append(appended)
            
        except:
            new_oprt.append(np.array([np.NaN]*4))

    new_oprt_whole.append(np.array(new_oprt).flatten())

In [35]:
new_col = list(columns.keys())
new_col.remove('OPRT_YMD')
new_col.append('OPRT_DIFF')

In [36]:
col = []
for i in range(1,5):
    for j in range(len(new_col)):
        col.append(new_col[j]+'_'+str(i))

record_df = pd.DataFrame(new_oprt_whole,columns=col) # snapshot data of patient who has record
pt_num  = record_df['PT_SBST_NO_1']

In [37]:
record_df = record_df[record_df.columns.drop(list(record_df.filter(regex='PT_SBST_NO')))]
#pt_num = 

In [38]:
#record_df['PT_SBST_NO'] = pt_num
record_df.insert(0, 'PT_SBST_NO', pt_num, allow_duplicates=False)

In [39]:
record_df

,PT_SBST_NO,OPRT_CLCN_OPRT_KIND_CD_1,OPRT_CURA_RSCT_CD_1,OPRT_DIFF_1,OPRT_CLCN_OPRT_KIND_CD_2,OPRT_CURA_RSCT_CD_2,OPRT_DIFF_2,OPRT_CLCN_OPRT_KIND_CD_3,OPRT_CURA_RSCT_CD_3,OPRT_DIFF_3,OPRT_CLCN_OPRT_KIND_CD_4,OPRT_CURA_RSCT_CD_4,OPRT_DIFF_4
0,RN00000006,12,NaN,283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RN00000008,1,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RN00000009,1,NaN,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RN00000012,1,2.0,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RN00000017,1,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549,RN00018846,99,2.0,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8550,RN00018847,11,NaN,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8551,RN00018850,11,2.0,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8552,RN00018860,11,2.0,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
bsnf[bsnf['PT_SBST_NO'].isin(oprt_nfrm['PT_SBST_NO'])]

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_FRST_DIAG_CD,BSPT_FRST_DIAG_NM,BSPT_IDGN_AGE,BSPT_FRST_ENDS_RSCT_YMD,...,BSPT_FRST_RDT_STRT_YMD,BSPT_DEAD_YMD,CENTER_LAST_VST_YMD,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT
0,90,4-2021-05-20,RN00000006,M,19760728,20180917,C20,rectum,42,NaN,...,NaN,NaN,20200703,656,p,1,1,0,0,2022-05-03
1,90,4-2021-05-20,RN00000008,M,19450219,20091102,C18,colon,64,NaN,...,NaN,NaN,20141111,1836,p,1,2,0,0,2022-05-03
2,90,4-2021-05-20,RN00000009,F,19391009,20121113,C18,colon,73,NaN,...,NaN,20140818.0,20140813,644,p,3,3,2,0,2022-05-03
3,90,4-2021-05-20,RN00000012,F,19470410,20150602,C18,colon,68,NaN,...,NaN,NaN,20210526,2186,p,3,3,1,0,2022-05-03
6,90,4-2021-05-20,RN00000017,F,19341209,20131212,C18,colon,79,NaN,...,NaN,NaN,20160824,987,p,3,3,1,0,2022-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10331,90,4-2021-05-20,RN00018846,F,19550806,20091228,C19,rectosigmoid junction,54,NaN,...,20120517.0,20130612.0,20130419,1263,p,3,3,1,0,2022-05-03
10332,90,4-2021-05-20,RN00018847,F,19570226,20140705,C20,rectum,57,20140705.0,...,NaN,NaN,20200616,2174,p,1,1,0,0,2022-05-03
10333,90,4-2021-05-20,RN00018850,F,19570817,20150902,C18,colon,58,NaN,...,NaN,NaN,20200805,1800,p,2,3,0,0,2022-05-03
10336,90,4-2021-05-20,RN00018860,F,19670325,20180825,C18,colon,51,NaN,...,NaN,NaN,20210316,935,p,3,3,2,0,2022-05-03


In [79]:
merged = pd.merge(bsnf[bsnf['PT_SBST_NO'].isin(oprt_nfrm['PT_SBST_NO'])],record_df, how='left',left_on='PT_SBST_NO',right_on ='PT_SBST_NO' )

In [ ]:
dead_no = dead[dead['PT_SBST_NO'].isin(merged['PT_SBST_NO'])]['PT_SBST_NO']

In [85]:
merged['DEAD'] = merged['PT_SBST_NO'].isin(dead_no).astype(int)

In [86]:
merged

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_FRST_DIAG_CD,BSPT_FRST_DIAG_NM,BSPT_IDGN_AGE,BSPT_FRST_ENDS_RSCT_YMD,...,OPRT_CLCN_OPRT_KIND_CD_2,OPRT_CURA_RSCT_CD_2,OPRT_DIFF_2,OPRT_CLCN_OPRT_KIND_CD_3,OPRT_CURA_RSCT_CD_3,OPRT_DIFF_3,OPRT_CLCN_OPRT_KIND_CD_4,OPRT_CURA_RSCT_CD_4,OPRT_DIFF_4,DEAD
0,90,4-2021-05-20,RN00000006,M,19760728,20180917,C20,rectum,42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,90,4-2021-05-20,RN00000008,M,19450219,20091102,C18,colon,64,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,90,4-2021-05-20,RN00000009,F,19391009,20121113,C18,colon,73,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,90,4-2021-05-20,RN00000012,F,19470410,20150602,C18,colon,68,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,90,4-2021-05-20,RN00000017,F,19341209,20131212,C18,colon,79,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549,90,4-2021-05-20,RN00018846,F,19550806,20091228,C19,rectosigmoid junction,54,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8550,90,4-2021-05-20,RN00018847,F,19570226,20140705,C20,rectum,57,20140705.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8551,90,4-2021-05-20,RN00018850,F,19570817,20150902,C18,colon,58,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8552,90,4-2021-05-20,RN00018860,F,19670325,20180825,C18,colon,51,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [87]:
merged = merged.drop(['CENTER_CD','IRB_APRV_NO','BSPT_BRYM','BSPT_FRST_DIAG_CD'],axis=1)

In [88]:
merged = merged.drop(merged.filter(like='YMD').columns,axis=1)

In [89]:
merged = merged.replace('x',np.nan)

In [90]:
from sklearn.preprocessing import LabelEncoder

encoders = []
for col in merged.columns:
    try:
        merged[col].astype(float)
    except:
        print(col)
        merged[col].astype(str)
        encoder = LabelEncoder()
        encoder.fit(merged[col])
        encoders.append(encoder)
        trans = encoder.transform(merged[col])
        merged[col] = trans

PT_SBST_NO
BSPT_SEX_CD
BSPT_FRST_DIAG_NM
BSPT_STAG_CLSF_CD
CRTN_DT


In [91]:
merged.to_csv('/home/dogu86/2022_DATA_SYNTHESIS/OPRT.csv')